# Classification with random forest

In [48]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #We need this to split the data
from sklearn.ensemble import RandomForestClassifier 

In [49]:
df = pd.read_csv('creditcard.csv')
df.dropna()
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Use RandomForest to predict Fraud
With Classification

28 Variabels that means something we dont now, but certainly transactions
### Amout
### Class (0 = no fraud, 1 = fraud)
### time (what does time mean?) Not interesting because it is very likly the time between two transaction

In [50]:
from sklearn.preprocessing import normalize #get the function needed to normalize our data.

X = df[['Amount', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5','V6','V7','V8','V9','V10','V11','V12','V13', 'V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28']] #create the X matrix
X = normalize(X) #normalize the matrix to put everything on the same scale
y = df['Class'] #create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40) #split the data, store it into different variables

from sklearn.neighbors import KNeighborsClassifier #the object class we need

knn = KNeighborsClassifier(n_neighbors=4) #create a KNN-classifier with 5 neighbors (default)
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data
knn.score(X_test, y_test) #calculate the fit on the test data

from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

#In order to read it easily , let's make a dataframe out of it, and add labels to it.
conf_matrix = pd.DataFrame(cm, index=['NoFraud', 'Fraud'], columns = ['NoFraud_p', 'Fraud_p']) 
conf_matrix

report = sk.metrics.classification_report(y_test, y_test_pred)
report

In [51]:
#random forest

randomf = RandomForestClassifier(n_estimators=100)
randomf = randomf.fit(X_train, y_train)
randomf.score(X_test, y_test)

0.9993

In [52]:
from sklearn.ensemble import RandomForestClassifier #the object class we need

knn = RandomForestClassifier(n_neighbors=2) #create a KNN-classifier with 5 neighbors (default)
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data
knn.score(X_test, y_test) #calculate the fit on the test data


0.9990333333333333

In [53]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[29925,     2],
       [   27,    46]], dtype=int64)

In [54]:
#In order to read it easily , let's make a dataframe out of it, and add labels to it.
conf_matrix = pd.DataFrame(cm, index=['NoFraud', 'Fraud'], columns = ['NoFraud_p', 'Fraud_p']) 
conf_matrix

,NoFraud_p,Fraud_p
NoFraud,29925,2
Fraud,27,46


In [55]:
report = sk.metrics.classification_report(y_test, y_test_pred)
report

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     29927\n           1       0.96      0.63      0.76        73\n\n    accuracy                           1.00     30000\n   macro avg       0.98      0.82      0.88     30000\nweighted avg       1.00      1.00      1.00     30000\n'